In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import os

os.chdir("/home/nacho/GNN4OPF/")

import funciones.parser  as ps

In [7]:
df_branch = ps.branch_data_df('data/prueba_sin_sobrecargas.raw')
df_bus = ps.bus_data_df('data/prueba_sin_sobrecargas.raw')
df_load = ps.load_data_df('data/prueba_sin_sobrecargas.raw')

df_load

,I,ID,STAT,AREA,ZONE,PL,QL,IP,IQ,YP,YQ,OWNER,SCALE,INTRPT,DGENP,DGENQ,DGENF
0,'ACO150_B 150.00','1 ',1,99,906,16.148,7.872,0.0,0.0,0.0,0.0,903,0,0,0.0,0.0,0
1,'ARA150_B 150.00','1 ',0,99,908,-3.366,0.620,0.0,0.0,0.0,0.0,901,1,0,0.0,0.0,0
2,'ARA150_B 150.00','2 ',0,99,908,-3.425,-0.591,0.0,0.0,0.0,0.0,901,1,0,0.0,0.0,0
3,'ARA150_B 150.00','G1',1,99,908,7.694,0.000,0.0,0.0,0.0,0.0,901,1,0,0.0,0.0,0
4,'ART150_B 150.00','1 ',1,99,908,4.121,-0.848,0.0,0.0,0.0,0.0,901,1,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,'VAL150_B 150.00','G1',1,99,905,1.491,-0.256,0.0,0.0,0.0,0.0,901,1,0,0.0,0.0,0
150,'YOU150_B 150.00','1 ',0,99,906,-8.711,0.000,0.0,0.0,0.0,0.0,901,1,0,0.0,0.0,0
151,'YOU150_B 150.00','2 ',0,99,906,-8.670,0.000,0.0,0.0,0.0,0.0,901,1,0,0.0,0.0,0
152,'YOU150_B 150.00','G1',1,99,906,6.913,0.000,0.0,0.0,0.0,0.0,901,1,0,0.0,0.0,0


In [8]:
def get_Admittances(branch_df):
  Admitance = np.empty((branch_df.shape[0],2),dtype = 'complex_')
  Admitance[:,0] = 1/(branch_df['R']+branch_df['X']*1j)
  Admitance[:,1] = branch_df['B']/2*1j
  Admitance = pd.DataFrame(Admitance,columns=['y','ysh'])

  Admitance['I'] = branch_df['I']
  Admitance['J'] = branch_df['J']
  Admitance = Admitance[['I','J','y','ysh']]
  return Admitance

def get_graph(edges_df):
  new_rows = []
  for _, row in edges_df.iterrows():
      source, sink, _, self_loop_weight = row
      new_rows.append({'source': source, 'target': source, 'weight': self_loop_weight})
      new_rows.append({'source': sink, 'target': sink, 'weight': self_loop_weight})
  new_edges_df = edges_df.drop(columns=['ysh'])
  new_edges_df.rename(columns={'I': 'source', 'J': 'target','y': 'weight'}, inplace=True)
  new_rows = pd.DataFrame(new_rows)
  new_edges_df = pd.concat([new_edges_df,new_rows], axis=0)
  new_edges_df_2 = new_edges_df.groupby(['source', 'target'])['weight'].sum().reset_index()
  G = nx.from_pandas_edgelist(new_edges_df_2, edge_attr=True)
  return G

def get_Ybus(G):
  Y = nx.laplacian_matrix(G).toarray()
  for i, node in enumerate(G.nodes()):
      degree = G.degree(node,weight='weight')
      Y[i, i] += (degree - Y[i, i])  / 2
  return Y

def get_Voltage(bus_data):
  bus_data = bus_data.set_index('NAME')
  bus_data.index = bus_data.index.str.replace('\'', '', regex=False)
  bus_data.index = bus_data.index.str.replace(' ', '', regex=False)
  bus_data['V (KV)'] = bus_data['BASKV'] * bus_data['VM'] * (np.cos(np.deg2rad(bus_data['VA'])) + np.sin(np.deg2rad(bus_data['VA'])) * 1j)
  bus_data = bus_data.reindex(nombres_ordenados)
  V = bus_data['V (KV)'].to_numpy()
  return bus_data['V (KV)']

In [4]:
Admitance = get_Admittances(df)
G = get_graph(Admitance)
Ybus = get_Ybus(G)

Ybus
# for edge in G.edges():
#     weight_complex = G.edges[edge]['weight']
#     weight_magnitude = abs(weight_complex)
#     G.edges[edge]['weight'] = weight_magnitude

# # Definir el layout y visualizar los nodos
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, node_size=1000, node_color='skyblue', font_size=10, font_color='darkblue', font_weight='bold')

# # Dibujar los enlaces con colores y grosores basados en los valores transformados
# nx.draw_networkx_edges(G, pos, edge_color='gray')

# # Agregar anotaciones o leyendas si es necesario

# # Mostrar la visualización
# plt.show()


array([[ 54.43848367 -184.82005592j, -27.21924183  +92.41190796j,
        -27.21924183  +92.41190796j, ...,   0.           +0.j        ,
          0.           +0.j        ,   0.           +0.j        ],
       [-27.21924183  +92.41190796j,  33.21687036 -112.6573518j ,
          0.           +0.j        , ...,   0.           +0.j        ,
          0.           +0.j        ,   0.           +0.j        ],
       [-27.21924183  +92.41190796j,   0.           +0.j        ,
         33.62390649 -114.03265439j, ...,   0.           +0.j        ,
          0.           +0.j        ,   0.           +0.j        ],
       ...,
       [  0.           +0.j        ,   0.           +0.j        ,
          0.           +0.j        , ...,   2.5896885    -7.76089491j,
         -2.5896885    +7.78106991j,   0.           +0.j        ],
       [  0.           +0.j        ,   0.           +0.j        ,
          0.           +0.j        , ...,  -2.5896885    +7.78106991j,
         20.7034422   -76.25221791j

In [9]:
get_Voltage(df_bus)

NameError: name 'nombres_ordenados' is not defined